# Async Server

In [6]:
import subprocess
import os
import signal
p=subprocess.Popen(["./start_server.sh"], preexec_fn=os.setsid)

In [ ]:
os.killpg(os.getpgid(p.pid), signal.SIGTERM)

# Server Client

In [7]:
#create client
from server.baseClient import dbClient
client = dbClient()

INFO:root:client: starting at 22222


In [18]:
#insert image, do prediction and insert label
print(client.predict_and_insert('test.jpeg'))

INFO:root:client: sending predict_and_insert request


True


In [10]:
#query
print(client.query("SELECT * FROM images"))

INFO:root:client: sending query


[('a9535674ec565c0e', 'server/data/images/other/a9535674ec565c0e.jpeg', 0, 5, 'other', 'NULL')]


In [19]:
print(client.query("SELECT * FROM modelLabels"))

INFO:root:client: sending query


[(1, 'testing', 'a9535674ec565c0e', 8, 0.0), (2, 'testing', 'a9535674ec565c0e', 3, 0.0)]


In [26]:
#get random. only work when there are weak labels
print(client.getRandomImageWithWeakLabel())

INFO:root:client: requesting weak label


{'labels': [1, 6], 'path': '/Users/wangmeili/Documents/sedma/server/data/images/other/bb4ce465d2962c8e.jpg', 'id': 'bb4ce465d2962c8e'}


In [16]:
#insert image but no prediction
print(client.insertImage('test.jpeg'))

a9535674ec565c0e duplicated entry, insert reverted


# Data Base Client

In [2]:
#create db instance
from server.database import databaseAPI
db=databaseAPI('server/test.db','server/data')

In [3]:
print(db.query_meta("SELECT COUNT(*) FROM modelLabels"))

[(93,)]


In [4]:
print(db.getRandomImageWithWeakLabel())

{'labels': [1, 6], 'id': 'bb4ce465d2962c8e', 'path': '/Users/wangmeili/Documents/sedma/server/data/images/other/bb4ce465d2962c8e.jpg'}


In [5]:
print(db.query_meta("SELECT id FROM images WHERE label=0"))

[('a9535674ec565c0e',), ('a15ee83ccb9231a7',), ('57ccab23b1a6cc4a',), ('61031bb2f5d90ec7',), ('5f81e81ac6e73139',), ('d319632c78c38e9b',), ('87968ec64c636a73',), ('b5a3164c2d1f93e1',), ('23646c9bd4e99c9a',), ('ef619b1616ba61c1',), ('f3610c9b63cc6699',), ('8f0738cf44f946e4',), ('87b6b3c339266319',), ('91b76e482678b867',), ('1fc9e53319c66d06',), ('67399c6d07499c36',), ('a308d6f2a763989d',), ('6b90386f90abc6f8',), ('bd3bc34db941e051',), ('af765cf05ca184a9',), ('2939dac7a46825ed',), ('774cd3f9102941af',), ('5749ada1a139b996',), ('358306fccdc1cbc6',), ('75f206439b182fe3',), ('ad6f411badd2881e',), ('5f93a9893939b238',), ('d78d60396f511b92',), ('bb4ce465d2962c8e',), ('99b20c3ef46c96cc',), ('99666ed8625ea42d',), ('3b61609ece08997f',), ('83d353786d8e868b',), ('d783c63423e7722c',), ('17e7e405143c7b5a',), ('e3e3850505727e6e',), ('1fcbd1511bab2625',), ('5b31b20eec093eec',), ('017d470df094f8f3',), ('8f654c9d771f8043',), ('d720fc0f07f1391c',), ('1db1a7854846dbe9',), ('770a90aa1d751bc7',), ('5b6e2713

In [6]:
db.insertMultipleImagesParallel('tempimages')

In [14]:
db.removeModelLabel(image_id='a9535674ec565c0e duplicated entry, insert reverted')

True

In [6]:
print(db.query_meta("SELECT path, id from images"))

[('data/images/other/00767f1797130301.jpeg', '00767f1797130301'), ('data/images/other/787e7c7878707070.jpeg', '787e7c7878707070'), ('data/images/other/3f3e3e3e3e3e3e20.jpeg', '3f3e3e3e3e3e3e20'), ('data/images/other/027e7e7e7e7e7000.jpeg', '027e7e7e7e7e7000'), ('data/images/other/007f7f7e60400000.jpeg', '007f7f7e60400000'), ('data/images/other/01ffffff1f1f0f00.jpeg', '01ffffff1f1f0f00')]


In [4]:
db.synchronize()

2017-03-16 21:33:24,946 - root - INFO - checking images
2017-03-16 21:33:24,949 - root - WARNING - 3f3e3e3e3e3e3e20 not in file system
2017-03-16 21:33:24,953 - root - WARNING - 00767f1797130301 not in file system
2017-03-16 21:33:24,956 - root - WARNING - 027e7e7e7e7e7000 not in file system
2017-03-16 21:33:24,958 - root - WARNING - 01ffffff1f1f0f00 not in file system
2017-03-16 21:33:24,960 - root - WARNING - 007f7f7e60400000 not in file system
2017-03-16 21:33:24,963 - root - WARNING - 787e7c7878707070 not in file system
2017-03-16 21:33:24,966 - root - INFO - checking models
2017-03-16 21:33:24,968 - root - INFO - Done


In [12]:
print(db.query_meta('SELECT * FROM images where id = "8080707efec1c3f7"')==[('8080707efec1c3f7', 'data/images/other/8080707efec1c3f7.jpg', 'NULL', 5, 'other', 'NULL')])

True


In [21]:
db.removeImage('007f7f7e60400000')

In [14]:
db.printSchemas()

images:
  0: id(TEXT PRIMATY KEY)
  1: path(TEXT)
  2: label(TEXT)
  3: confidence(INTEGER)
  4: source(TEXT)
  5: comment(TEXT)
modelLabels:
  0: id(INTEGER PRIMATY KEY)
  1: model(TEXT)
  2: image_id(TEXT)
  3: label(TEXT)
  4: confidence(REAL)
models:
  0: name(TEXT)
  1: path(TEXT)
  2: accuracy(REAL)


# Image

In [29]:
#create new image
from PIL import Image
im=Image.new("RGB", (512, 512))
data=[(i//3,0,i//3) for i in range(512)]
im.putdata(data)

print(database.utility.hashImage(im))
#im.show()

c000000000000000


In [8]:
# hash image
from PIL import Image
import imagehash
hashed = imagehash.phash(Image.open('test.jpeg'))
print(hashed)

a9535674ec565c0e
